<a href="https://colab.research.google.com/github/hemantkashniyal/EVA4/blob/master/notebooks/Session8/Session_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pwd

import torch
print(torch.cuda.is_available())

! git clone https://github.com/hemantkashniyal/EVA4.git || true
! git -C EVA4 pull
! cp -r /content/EVA4/eva4 .
! cp -r /content/EVA4/requirements.txt .

/home/jovyan/workspace/notebooks/Session8
False


In [2]:
%env PYTHONDONTWRITEBYTECODE=true
! pip install -r requirements.txt
    

env: PYTHONDONTWRITEBYTECODE=true


In [3]:
%matplotlib inline

import json
from environs import Env

from eva4.config.config import TrainingConfig
from eva4.common import reset_seed

env = Env()
env.read_env("./cifar10.experiment.env.txt", recurse=False)

config = TrainingConfig()
print("Experiment Config: ", json.dumps(config, default=lambda x: x.__dict__, sort_keys=False, indent=4))

Experiment Config:  {
    "cuda_available": false,
    "device": "cpu",
    "epochs": 100,
    "desired_accuracy": 80.0,
    "break_on_reaching_desired_accuracy": true,
    "consecutive_desired_accuracy": 3,
    "dataset": {
        "type": "CIFAR10",
        "dataset_root": "../data",
        "batch_size": 64,
        "shuffle": true,
        "num_workers": 1,
        "pin_memory": true,
        "input_size": [
            3,
            32,
            32
        ],
        "input_channel": 3,
        "input_dimension": [
            32,
            32
        ],
        "dataset_std": [
            0.247,
            0.2435,
            0.2616
        ],
        "dataset_mean": [
            0.4914,
            0.4822,
            0.4465
        ]
    },
    "scheduler": {
        "type": "MultiStepLR",
        "milestones": [
            10,
            20,
            35,
            50
        ],
        "gamma": 0.1
    },
    "optimizer": {
        "type": "SGD",
        "learn

In [5]:
from torchvision import transforms
from imageaug.transforms import Colorspace, RandomAdjustment, RandomRotatedCrop

dataset_std = config.dataset.dataset_std
dataset_mean = config.dataset.dataset_mean

# Define training transforms
train_transforms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(dataset_mean, dataset_std),
])
train_transforms = None

# Define test transforms
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(dataset_mean, dataset_std),
])
test_transforms = None

In [6]:
reset_seed()
from eva4.dataset.dataset_manager import DatasetManager

dataset = DatasetManager.get_dataset(config.dataset, train_transforms, test_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
from eva4.network.network_manager import NetworkManager

from eva4.network.cifar10.cifar10 import CIFAR10Net
network = CIFAR10Net(config.network)
NetworkManager.summarize(network, config.network, config.dataset)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
              ReLU-2           [-1, 16, 32, 32]               0
       BatchNorm2d-3           [-1, 16, 32, 32]              32
           Dropout-4           [-1, 16, 32, 32]               0
     InternalBlock-5           [-1, 16, 32, 32]               0
            Conv2d-6           [-1, 32, 32, 32]           4,608
              ReLU-7           [-1, 32, 32, 32]               0
       BatchNorm2d-8           [-1, 32, 32, 32]              64
           Dropout-9           [-1, 32, 32, 32]               0
    InternalBlock-10           [-1, 32, 32, 32]               0
           Conv2d-11           [-1, 64, 30, 30]          18,432
             ReLU-12           [-1, 64, 30, 30]               0
      BatchNorm2d-13           [-1, 64, 30, 30]             128
          Dropout-14           [-1, 64,

/home/jovyan/workspace/notebooks/Session8/eva4/network/cifar10/cifar10.py:66: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [9]:
from eva4.optimizer.optimizer_manager import OptimizerManager

optimizer = OptimizerManager.get_optimizer(network, config.optimizer)

In [11]:
from eva4.scheduler.scheduler_manager import SchedulerManager

scheduler = SchedulerManager.get_scheduler(optimizer, config.scheduler)

In [12]:
from eva4.training.training_manager import TrainingManager


training = TrainingManager(config, dataset, network, scheduler, optimizer)
training.start()

  0%|          | 0/782 [00:00<?, ?it/s]

EPOCH: 0


Loss=2.405003070831299 Batch_id=2 Accuracy=8.33:   0%|          | 3/782 [00:06<31:35,  2.43s/it] Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 